<a href="https://colab.research.google.com/github/AnudeepReddy-Katta/RNNs-LSTM-Attention/blob/main/Part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [295]:
import pandas as pd
datasetSentences = pd.read_csv(("/content/drive/MyDrive/NLP/datasetSentences.txt"), sep="\t", skiprows=[0], names=['sentence_index','sentence'])
sentiment_labels = pd.read_csv(("/content/drive/MyDrive/NLP/sentiment_labels.txt"), sep="|", skiprows=[0], names=['phrase_id','sentiment_values'])
dictionary = pd.read_csv(("/content/drive/MyDrive/NLP/dictionary.txt"), sep="|", names=['phrase','phrase_id'])

In [296]:
def pre_process_sentences(string):
  string=string.replace('-LRB-','(')
  string=string.replace('-RRB-',')')
  string=string.replace('Â', '')
  string=string.replace('Ã©', 'e')
  string=string.replace('Ã¨', 'e')
  string=string.replace('Ã¯', 'i')
  string=string.replace('Ã³', 'o')
  string=string.replace('Ã´', 'o')
  string=string.replace('Ã¶', 'o')
  string=string.replace('Ã±', 'n')
  string=string.replace('Ã¡', 'a')
  string=string.replace('Ã¢', 'a')
  string=string.replace('Ã£', 'a')
  string=string.replace('\xc3\x83\xc2\xa0', 'a')
  string=string.replace('Ã¼', 'u')
  string=string.replace('Ã»', 'u')
  string=string.replace('Ã§', 'c')
  string=string.replace('Ã¦', 'ae')
  string=string.replace('Ã­', 'i')
  string=string.replace('\xa0', ' ')
  string=string.replace('\xc2', '')
  return string

In [297]:
datasetSentences['sentence'] = datasetSentences['sentence'].apply(pre_process_sentences)

In [298]:
def pre_process_phrases(string):
    string=string.replace('é','e')
    string=string.replace('è','e')
    string=string.replace('ï','i')
    string=string.replace('í','i')
    string=string.replace('ó','o')
    string=string.replace('ô','o')
    string=string.replace('ö','o')
    string=string.replace('á','a')
    string=string.replace('â','a')
    string=string.replace('ã','a')
    string=string.replace('à','a')
    string=string.replace('ü','u')
    string=string.replace('û','u')
    string=string.replace('ñ','n')
    string=string.replace('ç','c')
    string=string.replace('æ','ae')
    string=string.replace('\xa0', ' ')
    string=string.replace('\xc2', '')    
    return string

In [299]:
dictionary['phrase'] = dictionary['phrase'].apply(pre_process_phrases)

In [300]:
df1 = pd.merge(sentiment_labels,dictionary,on='phrase_id')
df1.drop(columns=['phrase_id'],inplace = True)
df1.head()

,sentiment_values,phrase
0,0.50000,!
1,0.50000,'
2,0.44444,' (
3,0.50000,' ( the cockettes
4,0.42708,' ( the cockettes )


In [301]:
df2 = pd.merge(df1,datasetSentences,left_on='phrase',right_on='sentence')
df2.drop(columns = ['phrase','sentence_index'],inplace = True)
df2.head()

,sentiment_values,sentence
0,0.013889,... a bland murder-on-campus yawner .
1,0.263890,... a hollow joke told by a cinematic gymnast ...
2,0.472220,... the picture 's cleverness is ironically mu...
3,0.875000,classic cinema served up with heart and humor
4,0.319440,"entertaining enough , but nothing new"


In [302]:
df2.loc[(df2['sentiment_values']>=0) & (df2['sentiment_values']<=0.2),'label']=0
df2.loc[(df2['sentiment_values']>0.2) & (df2['sentiment_values']<=0.4),'label']=1
df2.loc[(df2['sentiment_values']>0.4) & (df2['sentiment_values']<=0.6),'label']=2
df2.loc[(df2['sentiment_values']>0.6) & (df2['sentiment_values']<=0.8),'label']=3
df2.loc[(df2['sentiment_values']>0.8) & (df2['sentiment_values']<=1),'label']=4
df2.drop(columns = ['sentiment_values'],inplace = True)
df2.rename(columns={'label':'sentiment_values'},inplace = True)
df2['sentiment_values'] = df2['sentiment_values'].astype('int')

In [303]:
df2['sentiment_values'].value_counts()

1    3140
3    3111
2    2242
4    1851
0    1510
Name: sentiment_values, dtype: int64

In [304]:
len(df2)

11854

In [305]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df2, test_size=0.3, random_state=42, stratify=df2['sentiment_values'])

In [306]:
train = train.reset_index()
train.drop(columns='index',inplace=True)
test =test.reset_index(drop=True)

In [307]:
# Import Library
import random
import torch, torchtext
from torchtext.legacy import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [308]:
Sentence = data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [309]:
fields = [('sentences', Sentence),('labels',Label)]

In [310]:
example = [data.Example.fromlist([train.sentence[i],train.sentiment_values[i]], fields) for i in range(train.shape[0])]

In [311]:
train_data = data.Dataset(example, fields)

In [312]:
example_test = [data.Example.fromlist([test.sentence[i],test.sentiment_values[i]], fields) for i in range(test.shape[0])]

In [313]:
valid_data = data.Dataset(example_test, fields)

In [314]:
(len(train_data), len(valid_data))

(8297, 3557)

In [315]:
vars(valid_data.examples[50])

{'labels': 2,
 'sentences': ['It',
  'will',
  'break',
  'your',
  'heart',
  'many',
  'times',
  'over',
  '.']}

In [316]:
Sentence.build_vocab(train_data)
Label.build_vocab(train_data)

In [317]:
print('Size of input vocab : ', len(Sentence.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Sentence.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  16965
Size of label vocab :  5
Top 10 words appreared repeatedly : [('.', 7814), (',', 6948), ('the', 5839), ('and', 4266), ('a', 4237), ('of', 4235), ('to', 3023), ('-', 2654), ("'s", 2496), ('is', 2422)]
Labels :  defaultdict(None, {1: 0, 3: 1, 2: 2, 4: 3, 0: 4})


In [318]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [319]:
train_iterator, valid_iterator = data.BucketIterator.splits((train_data, valid_data), batch_size = 64, 
                                                            sort_key = lambda x: len(x.sentences),
                                                            sort_within_batch=True, 
                                                            device = device)

In [320]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(sentence.vocab.stoi, tokens)

In [321]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True,
                           bidirectional=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [322]:
# Define hyperparameters
size_of_vocab = len(Sentence.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 5
num_layers = 2
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [323]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(16965, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=100, out_features=5, bias=True)
)
The model has 5,653,205 trainable parameters


In [324]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

# define metric
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [325]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        sent, sent_lengths = batch.sentences   
        
        # convert to 1D tensor
        predictions = model(sent, sent_lengths).squeeze()  
        # compute the loss
        loss = criterion(predictions, batch.labels)        
        
        # compute the binary accuracy
        acc = categorical_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [326]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            sent, sent_lengths = batch.sentences
            
            # convert to 1d tensor
            predictions = model(sent, sent_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)
            acc = categorical_accuracy(predictions, batch.labels)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [327]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 1.575 | Train Acc: 28.42%
	 Val. Loss: 1.560 |  Val. Acc: 31.57% 

	Train Loss: 1.526 | Train Acc: 36.89%
	 Val. Loss: 1.534 |  Val. Acc: 35.08% 

	Train Loss: 1.441 | Train Acc: 45.76%
	 Val. Loss: 1.524 |  Val. Acc: 36.26% 

	Train Loss: 1.347 | Train Acc: 56.49%
	 Val. Loss: 1.520 |  Val. Acc: 36.46% 

	Train Loss: 1.259 | Train Acc: 65.64%
	 Val. Loss: 1.528 |  Val. Acc: 35.84% 



In [340]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_sentence(sentence):
    
    categories = {0: "very negative", 1:"negative", 2:"neutral", 3:"positive", 4:"very positive"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    pred = prediction.argmax(1)
    
    return categories[pred.item()]

In [354]:
for i in range(10):
  print(test.sentence[i])
  print("predicted: ", classify_sentence(test.sentence[i]))
  print("actual: ", test.sentiment_values[i])
  print('\n')

The performances are so leaden , Michael Rymer 's direction is so bloodless and the dialogue is so corny that the audience laughs out loud .
predicted:  very negative
actual:  1


Lacks the inspiration of the original and has a bloated plot that stretches the running time about 10 minutes past a child 's interest and an adult 's patience .
predicted:  very negative
actual:  0


Works because , for the most part , it avoids the stupid cliches and formulaic potholes that befall its brethren .
predicted:  very negative
actual:  3


Cox is far more concerned with aggrandizing madness , not the man , and the results might drive you crazy .
predicted:  neutral
actual:  2


Stanley Kwan has directed not only one of the best gay love stories ever made , but one of the best love stories of any stripe .
predicted:  negative
actual:  4


Grown-up quibbles are beside the point here .
predicted:  very negative
actual:  2


Though the book runs only about 300 pages , it is so densely packed ... that